In [1]:
import json
import os
from tqdm import tqdm

In [2]:
triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
mappings_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
filtered_triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples/"
filtered_triples_weighted_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_weighted/"

In [3]:
#edges_to_keep = ["http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLink", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://purl.org/dc/terms/subject", "http://www.w3.org/2004/02/skos/core#broader"]
edges_to_keep = ["http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLink"]
edges_to_delete = ["http://dbkwik.webdatacommons.org/ontology/Image", "http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLinkText", "/File:", "/Special:FilePath/"]

In [4]:
triples = sorted([elem for elem in os.listdir(triples_path) if ".triples" in elem])
mappings = sorted([elem for elem in os.listdir(triples_path) if ".json" in elem])
filtered_triples = sorted([elem for elem in os.listdir(filtered_triples_path) if ".triples" in elem])
filtered_triples_weighted = sorted([elem for elem in os.listdir(filtered_triples_weighted_path) if ".triples" in elem])
num_of_files = len(triples)

In [5]:
for i in range(num_of_files):

    triple = triples[i]
    mapping = mappings[i]

    kept = 0
    deleted = 0

    with open(triples_path + triple, "r") as t, open(mappings_path + mapping, "r") as m, open(filtered_triples_path + triple, "w") as f:

        mapping = json.load(m)
        reversed_mapping = {str(v): k for k, v in mapping.items()}

        for line in t:
            entities = line.strip().split("###")
            edge_type = reversed_mapping.get(entities[2], None)
            if edge_type not in edges_to_keep:
                deleted += 1
                continue
            node_1 = reversed_mapping.get(entities[0], None)
            node_2 = reversed_mapping.get(entities[1], None)
            line_as_string = node_1 + "###" + node_2
            if any(substring in line_as_string for substring in edges_to_delete):
                deleted += 1
            else:
                kept += 1
                f.write(line)
                
        print(f"{triple}: {deleted}/{kept + deleted} triples are deleted")

KeyboardInterrupt: 

In [ ]:
for i in range(num_of_files):

    triple = triples[i]
    filtered_triple = filtered_triples[i]
    nodes = set()
    nodes_filtered = set()
    lines = 0
    lines_filtered = 0

    with open(triples_path + triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            nodes.add(triple_list[0])
            nodes.add(triple_list[1])

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines_filtered += 1
            triple_list = line.replace("\n", "").split("###")
            nodes_filtered.add(triple_list[0])
            nodes_filtered.add(triple_list[1])

    print(f"{triple}: {len(nodes)} nodes, {lines} edges --> {len(nodes_filtered)} nodes, {lines_filtered} edges")

In [ ]:
for i in range(num_of_files):
    
    print("Processing file", i + 1, "of", num_of_files)

    filtered_triple = filtered_triples[i]
    edges = dict()
    lines = 0

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = sorted(line.replace("\n", "").split("###")[:2])
            key = str(triple_list[0]) + "###" + str(triple_list[1])
            edges[key] = edges.get(key, 0) + 1

    with open(filtered_triples_weighted_path + filtered_triple, "w") as fw:
        for k, v in edges.items():
            fw.write(str(k) + "###" + str(v) + "\n")

In [ ]:
for i in range(num_of_files):

    triple = triples[i]
    filtered_triple_weighted = filtered_triples_weighted[i]
    nodes = set()
    nodes_filtered_weighted = set()
    lines = 0
    lines_filtered_weighted = 0

    with open(triples_path + triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            nodes.add(triple_list[0])
            nodes.add(triple_list[1])

    with open(filtered_triples_weighted_path + filtered_triple_weighted, "r") as t:
        for line in t:
            lines_filtered_weighted += 1
            triple_list = line.replace("\n", "").split("###")
            nodes_filtered_weighted.add(triple_list[0])
            nodes_filtered_weighted.add(triple_list[1])

    print(f"{triple}: {len(nodes)} nodes, {lines} edges --> {len(nodes_filtered_weighted)} nodes, {lines_filtered_weighted} edges")